# 원, 삼각형, 사각형 구분하기

데이터 저장소에 저장 후 

In [1]:
!unzip -qq handwriting_shape.zip -d handwriting_shape

In [2]:
!unzip -qq hard_handwriting_shape.zip -d hard_handwriting_shape

## 3.Data Augumentation을 사용한 CNN 모델

In [6]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import *
from keras.preprocessing.image import ImageDataGenerator

In [7]:
# seed 값 설정
seed = 2021
np.random.seed(seed)
tf.random.set_seed(seed)

### 데이터 부풀리기

In [9]:
from keras.preprocessing.image  import *

In [11]:
# 데이터 부풀리기를 적용한 훈련 데이터 제너레이터
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=15,
                                   width_shift_range=0.1,
                                   height_shift_range=0.1,
                                   shear_range=0.5,
                                   zoom_range=[0.5,2.0],
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                   fill_mode='nearest')

In [ ]:
!mkdir preview

In [19]:
img = load_img('hard_handwriting_shape/train/triangle/triangle001.png')
x = img_to_array(img)
x = x.reshape((1,) + x.shape)

i = 0

# 무한 for loop이니 주의!!!
for batch in train_datagen.flow(x, batch_size=1, save_to_dir='preview', 
                                save_prefix='tri', save_format='png'):
    i += 1
    if i > 30: 
        break

In [20]:
!ls preview

tri_0_1302.png	tri_0_4932.png	tri_0_5881.png	tri_0_8211.png	tri_0_9797.png
tri_0_2407.png	tri_0_4949.png	tri_0_6416.png	tri_0_8288.png	tri_0_9854.png
tri_0_2997.png	tri_0_5158.png	tri_0_6427.png	tri_0_8923.png	tri_0_9972.png
tri_0_3418.png	tri_0_519.png	tri_0_7039.png	tri_0_9089.png
tri_0_3503.png	tri_0_5300.png	tri_0_7259.png	tri_0_928.png
tri_0_3759.png	tri_0_5672.png	tri_0_7282.png	tri_0_9495.png
tri_0_3825.png	tri_0_5798.png	tri_0_7897.png	tri_0_9575.png


### 데이터셋 생성하기

In [21]:
train_generator = train_datagen.flow_from_directory(
    'hard_handwriting_shape/train',
    target_size=(24,24),
    batch_size=3,
    class_mode = 'categorical')

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    'hard_handwriting_shape/test',
    target_size=(24,24),
    batch_size=3,
    class_mode = 'categorical')

Found 45 images belonging to 3 classes.
Found 15 images belonging to 3 classes.


In [22]:
train_generator.labels

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2], dtype=int32)

In [23]:
train_generator.filenames[0]

'circle/circle001.png'

### 모델 구성하기

In [28]:
model = Sequential([
                    Conv2D(32, kernel_size=(3,3),
                           activation='relu', input_shape=(24,24,3)),
                    Conv2D(64,(3,3),activation='relu'),
                    MaxPooling2D(),
                    Flatten(),
                    Dense(128,activation='relu'),
                    Dense(3,activation='softmax')
])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 22, 22, 32)        896       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 20, 20, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 10, 10, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 6400)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               819328    
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 387       
Total params: 839,107
Trainable params: 839,107
Non-trainable params: 0
________________________________________________

### 모델 학습과정 설정하기

In [29]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',metrics=['accuracy'])

### 모델 학습시키기
- 첫번째 인자 : 훈련데이터셋을 제공할 제네레이터를 지정. 본 예제에서는 앞서 생성한 train_generator으로 지정.
- steps_per_epoch : 한 epoch에 사용한 스텝 수를 지정. 총 45개의 훈련 샘플이 있고 배치사이즈가 3이므로 15 스텝으로 지정.
- epochs : 전체 훈련 데이터셋에 대해 학습 반복 횟수를 지정. 50번을 반복적으로 학습.
- validation_data : 검증데이터셋을 제공할 제네레이터를 지정. 본 예제에서는 앞서 생성한 validation_generator으로 지정.
- validation_steps : 한 epoch 종료 시 마다 검증할 때 사용되는 검증 스텝 수를 지정. 홍 15개의 검증 샘플이 있고 배치사이즈가 3이므로 5 스텝으로 지정.

### 모델 학습

In [30]:
model.fit_generator(
    train_generator,
    steps_per_epoch=15,
    epochs=200,
    validation_data=test_generator,
    validation_steps=5
)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/200
15/15 [==============================] - 1s 36ms/step - loss: 1.8253 - accuracy: 0.2968 - val_loss: 1.0974 - val_accuracy: 0.4000
Epoch 2/200
15/15 [==============================] - 0s 25ms/step - loss: 1.0949 - accuracy: 0.4265 - val_loss: 1.0779 - val_accuracy: 0.3333
Epoch 3/200
15/15 [==============================] - 0s 25ms/step - loss: 1.0833 - accuracy: 0.3145 - val_loss: 1.1010 - val_accuracy: 0.3333
Epoch 4/200
15/15 [==============================] - 0s 24ms/step - loss: 1.0759 - accuracy: 0.3442 - val_loss: 1.0876 - val_accuracy: 0.2667
Epoch 5/200
15/15 [==============================] - 0s 24ms/step - loss: 0.9555 - accuracy: 0.6208 - val_loss: 0.9948 - val_accuracy: 0.3333
Epoch 6/200
15/15 [==============================] - 0s 30ms/step - loss: 0.8751 - accuracy: 0.6154 - val_loss: 0.9761 - val_accuracy: 0.4667
Epoch 7/200
15/15 [==============================] - 0s 24ms/step - loss: 0.8415 - accuracy: 0.6342 - val_loss: 0.9822 - val_accuracy: 0.3333
Epoch 

### 모델 평가

In [33]:
print("-- Evaluate --")
scores = model.evaluate_generator(test_generator, steps=5)
print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))

-- Evaluate --
accuracy: 60.00%


/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1877: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '
